# Notebook para ir probando código

### Prueba de calculadora de energía de una secuencia de aminoácidos

In [1]:
"""
countH(HPlist)

Given a vector (H,H,P,H,P,...)=(-1,-1,1,-1,1,...), counts the number of H-H bonds within the array.
"""
function countH(HPlist)
    s=0
    for i in 1:(length(HPlist)-1)
        if HPlist[i] == 1
            continue
        elseif HPlist[i] == -1
            if HPlist[i+1] == 1
                continue
            elseif HPlist[i+1] == -1
                s=s+1
            end
        end
    end
    return s
end

# It works as expected

countH([-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1])

7

In [2]:
"""
energy(N,edo,HPlist)

Given a an array side N, a matrix encoding the aminoacids positions, and an array containing the sequence of H,P aminoacids; the function
outputs the energy of the configuration.
"""
function energy(N,edo,HPlist)
    red=zeros(Int8,(N,N))

    # I set up the protein within the array.
    for k in 1:length(HPlist)
        x=edo[k,1]
        y=edo[k,2]
        red[x,y]=HPlist[k]
    end
    
    
    # I create an auxiliary array with periodic boundary conditions.
    redaux=zeros(Int8,(N+2,N+2))
    redaux[2:end-1,2:end-1]=red
    redaux[end,2:end-1]=red[1,:]
    redaux[1,2:end-1]=red[end,:]
    redaux[2:end-1,end]=red[:,1]
    redaux[2:end-1,1]=red[:,end]
    
    #display(redaux)

    # Next, I iterate over the protein´s vertices and compute the number of H-H bonds.
    # To avoid counting the covalent bonds, I substract two times the value of `countBH(HPlist)`.
    enp=0
    for i in 1:length(HPlist)
        if HPlist[i]==1 
            continue
        elseif HPlist[i]==-1
            x=edo[i,1]+1
            y=edo[i,2]+1 # I have to shift both indice by one place because we are working on the extended array `redaux`.
            cruz=[redaux[x+1,y],redaux[x,y+1],redaux[x-1,y],redaux[x,y-1]] # I define an array with the next closest neighbors.
            c=count(i->(i==-1),cruz)
            enp=enp+(c)
        end
    end
    en=((enp)-2*countH(HPlist))/2 # The total energy is obtained by substracting the number of covalent bonds to `enp` 
    #and dividing the resulting number by two.
    return en
end

# Funciona bien la función

energy(10,[[4 4];[5 4];[5 5];[4 5];[3 5];[3 6];[2 6];[2 5];[2 4];[3 4];[3 3]],[-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1])

4.0

### Prueba de indices peiódicos

In [3]:
# Now, I want a function which takes an array and two indices, and returns the indices taking periodic b.c. into account

"""
    perriodicInd2D(A,indices)

Given a 2D array `A` and a couple of indices, returns the indices for the equivalent array with periodic boundary conditions.
"""
function perriodicInd2D(A,indices)
    lx,ly=size(A)
    ix,iy=indices
    Ix=0
    Iy=0
    mx=mod(ix,lx)
    my=mod(iy,ly)
    
    # The following conditional imposes periodic boundary conditions by use of the `mod()` function.
    if mx != 0
        Ix=mx
    else
        Ix=lx
    end
    
    if my != 0
        Iy=my
    else
        Iy=ly
    end
    
    return ([Ix,Iy])
end

#Pruebo la función

mpp=rand(Int64,(3,3))
display(mpp)
ind1,ind2=perriodicInd2D(mpp,[3,2])
mpp[ind1,ind2] == mpp[3,2]

3×3 Matrix{Int64}:
 -7529316963305340593  2269115227269326801   3979699031640824801
  7332070554163025861  4248043218967491501   6712915092395804627
 -2645398375938493463  6905288346468852977  -6751972255004110072

true

### Prueba de función que comprueba la validez de una configuración.

In [7]:
# Since I end up setting uo the array wchich contains the protein sequence multiple times, I write a function to do 
#just that.
"""
    makeLattice(N,edo,HPlist)

Given a value for the lattice size `N`, a matrix encoding the aminoacids positions `edo`, and an array containing 
the sequence of H,P aminoacids `HPlist`; creates a 2D array containing the amino acid sequence.
"""
function makeLattice(N,edo,HPlist)
    red=zeros(Int8,(N,N))
    for k in 1:length(HPlist)
        x=edo[k,1]
        y=edo[k,2]
        red[x,y]=HPlist[k]
    end
    return red
end


"""
    validConf(N,ind,edo,HPlist)

Given a 2D array of side `N`, a matrix encoding the aminoacids positions `edo`, an index `ind` , and an array containing 
the sequence of H,P aminoacids `HPlist`; determines whether the protein structure is valid or not.
"""
function validConf(N,ind,edo,HPlist)
    
    # I set up the protein within the array.
    red=makeLattice(N,edo,HPlist)
    
    # Next, I iterate over the protein´s vertices, computing the distance to the next vertix.
    ans=true
    if ind !=1
        for j in ind:-1:2
            x1,y1=edo[j,:]
            x2,y2=edo[j-1,:]
            dx=abs(x1-x2)
            dy=abs(y1-y2)
            if (dx > 1 || dy > 1) == true # The sequence is not valid
                ans=false
                break
            else 
                continue
            end
        end
    end
return ans
end


validConf(10,8,[[4 4];[5 4];[6 4];[4 5];[3 5];[3 6];[2 6];[2 5];[2 4];[3 4];[3 3]],[-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1])

false

### Prueba de función que mueve un solo punto dentro de un array.

In [8]:
"""
    singleMove2D(arr,pos1,pos2)

Given a 2D array `arr`, an initial position  `pos1` ([xᵢ,yᵢ]) and a final desired position `pos2` ([xₛ,yₛ]); moves the value of 
# `arr[xᵢ,yᵢ]` to  `arr[xₛ,yₛ]`.
"""
eddo=[[4 4];[5 4];[5 5];[4 5];[3 5];[3 6];[2 6];[2 5];[2 4];[3 4];[3 3]]
HPplist=[-1,1,1,-1,-1,-1,-1,-1,-1,-1,-1]
redd=zeros(Int8,(10,10))

for k in 1:length(HPplist)
    x=eddo[k,1]
    y=eddo[k,2]
    redd[x,y]=HPplist[k]
end

display(redd)

function singleMove2D(arr,pos1,pos2)
    red=arr
    xi,yi=pos1
    xf,yf=pos2
    if pos1 != pos2
        red[xf,yf]=arr[xi,yi]
        red[xi,yi]=0
    end
    return red
end

# lA función funciona bien

display(singleMove2D(redd,[5,5],[6,6]))

10×10 Matrix{Int8}:
 0  0   0   0   0   0  0  0  0  0
 0  0   0  -1  -1  -1  0  0  0  0
 0  0  -1  -1  -1  -1  0  0  0  0
 0  0   0  -1  -1   0  0  0  0  0
 0  0   0   1   1   0  0  0  0  0
 0  0   0   0   0   0  0  0  0  0
 0  0   0   0   0   0  0  0  0  0
 0  0   0   0   0   0  0  0  0  0
 0  0   0   0   0   0  0  0  0  0
 0  0   0   0   0   0  0  0  0  0

10×10 Matrix{Int8}:
 0  0   0   0   0   0  0  0  0  0
 0  0   0  -1  -1  -1  0  0  0  0
 0  0  -1  -1  -1  -1  0  0  0  0
 0  0   0  -1  -1   0  0  0  0  0
 0  0   0   1   0   0  0  0  0  0
 0  0   0   0   0   1  0  0  0  0
 0  0   0   0   0   0  0  0  0  0
 0  0   0   0   0   0  0  0  0  0
 0  0   0   0   0   0  0  0  0  0
 0  0   0   0   0   0  0  0  0  0

### Prueba de un solo pull move en 2D

In [18]:

# Knowing how to asses the validity of a configuration, I proceed to write a function which performs a complete pull-move for all 
# of the amino acids within the protein chain, except for those at the ends
"""
    pullMove2D(N,ind,edo,HPlist)

Given a 2D array of side `N`, and index `ind`, a matrix encoding the aminoacids positions `edo`, and an array containing 
the sequence of H,P aminoacids `HPlist`; chooses a random vertex to pull. 
The functions stops when a valid configuration has been achieved.
"""
function pullMove2D(N,ind,edo,HPlist)
    
    # I set up the protein within the array.
    red=makeLattice(N,edo,HPlist)
    display(red)
    
    newedo=copy(edo)
    
    # Now we check wether a pull-move is possible for the selected vertex. A diagonal space must be empty. Also, the empty
    # diagonal space must be adjacent to the amino acid correpsonding to `ind+1`.
    # First I check where `ind+1` is.
    
    xplus=edo[ind+1,1]
    yplus=edo[ind+1,2] # Position of `ind+1`
    x=edo[ind,1]
    y=edo[ind,2] # Position of `ind`
    dx=x-xplus
    dy=y-yplus
    rel1= dx > 0 && dy == 0
    rel2= dx == 0 && dy < 0
    rel3= dx < 0 && dy == 0
    rel4= dx == 0 && dy > 0

    diagspaces=ones(Int8,4)
    if rel1 == true
        diagspaces[1]=red[x-1,y+1]
        diagspaces[4]=red[x-1,y-1]
    elseif rel2 == true
        diagspaces[1]=red[x-1,y+1]
        diagspaces[2]=red[x+1,y+1]
    elseif rel3 == true
        diagspaces[2]=red[x+1,y+1]
        diagspaces[3]=red[x+1,y-1]
    elseif rel4 == true
        diagspaces[3]=red[x+1,y-1]
        diagspaces[4]=red[x-1,y-1]
    end


    v1=red[x-1,y]
    v2=red[x,y+1]
    v3=red[x+1,y]
    v4=red[x,y-1]  # Possible sites to move `ind-1` to.

    cond1= edo[ind-1,:] == [x-1,y]
    cond2= edo[ind-1,:] == [x,y+1]
    cond3= edo[ind-1,:] == [x+1,y]
    cond4= edo[ind-1,:] == [x,y-1] # `edo[ind-1,:]` is the current location on the array `red` of `ind-1`.
    
    condi1= (v1 == 0) || cond1 # Either the place is empty, or `ind-1` is already there.
    condi2= (v2 == 0) || cond2
    condi3= (v3 == 0) || cond3
    condi4= (v4 == 0) || cond4
    condis=Bool[condi1 || condi2, condi2 || condi3, condi3 || condi4, condi4 || condi1]



    indices=Int8[]
    for k in 1:4
        space=diagspaces[k]
        # The diagonal space is empty, and the adjacent space is either empty or occupied by `ind-1` ,
        # pull move might be possible.
        if (space == 0 && condis[k] ) == true 
            push!(indices,k)
        else
            continue
        end
    end


    # I have an array with the possible spaces to move to. Now I have to check whether there is space for the amino acid 
    # correpsonding to `ind-1` to move to.  If `indices` is empty, the next part is ommited.
    if isempty(indices) == false

        xys=zeros(Int8,2) # To be filled with the position `ind-1` might be needed to be moved to.

        for el in indices

            if el == 1
                if condi1 == true # Either `v1` is empty or it is already occupied by `ind-1`. A pull move is all but assured.
                    singleMove2D(red,[x,y],[x-1,y+1]) # Make the move.
                    newedo[ind,:]=[x-1,y+1] # Record the change.
                    xys[:]=[x-1,y]
                    break
                elseif condi2 == true
                    singleMove2D(red,[x,y],[x-1,y+1])
                    newedo[ind,:]=[x-1,y+1]
                    xys[:]=[x,y+1]
                    break
                end


            elseif el == 2
                if condi2 == true
                    singleMove2D(red,[x,y],[x+1,y+1])
                    newedo[ind,:]=[x+1,y+1]
                    xys[:]=[x,y+1]
                    break
                elseif condi3 == true
                    singleMove2D(red,[x,y],[x+1,y+1])
                    newedo[ind,:]=[x+1,y+1]
                    xys[:]=[x+1,y]
                    break  
                end


            elseif el == 3 
                if condi3 == true
                    singleMove2D(red,[x,y],[x+1,y-1])
                    newedo[ind,:]=[x+1,y-1]
                    xys[:]=[x+1,y]
                    break
                elseif condi4 == true
                    singleMove2D(red,[x,y],[x+1,y-1])
                    newedo[ind,:]=[x+1,y-1]
                    xys[:]=[x,y-1]
                    break
                end


            elseif el == 4 
                if condi4 == true
                    singleMove2D(red,[x,y],[x-1,y-1])
                    newedo[ind,:]=[x-1,y-1]
                    xys[:]=[x,y-1]
                    break
                elseif condi1 == true
                    singleMove2D(red,[x,y],[x-1,y-1])
                    newedo[ind,:]=[x-1,y-1]
                    ys[:]=[x-1,y]
                    break
                end
            end
        end
        
        # Now I have in `xys` the potential new position for `ind-1`. If the current configuration is valid, I don´t make the move
        # and the full pull move has been succesfully completed.
        if validConf(N,ind,newedo,HPlist) != true
            singleMove2D(red,edo[ind-1,:],xys) # Makes the move.
            newedo[ind-1,:]=xys[:] # Records the change.
        end

    end
    

    # If all went well, a move has been done, now I have to check whether the current configuration is valid.
    stateconf=validConf(N,ind-1,newedo,HPlist) # `true` means the sequence is valid.
    for k in ind:-1:3
        if stateconf == false
            op=edo[k-2,:] # Old position.
            np=edo[k,:] # New position.
            singleMove2D(red,op,np) 
            newedo[k-2,:]=np 
            stateconf=validConf(N,k-2,newedo,HPlist) # Check whether the new configuration is valid.
        else
            break
        end
    end

    return (red,newedo,HPlist) # Returns everything neccesary to reproduce the final configuration.
end

estpp=[[6 3];[6 4];[6 5];[5 5];[5 6];[4 6];[4 7];[5 7];[6 7];[6 6];[7 6];[8 6]]
HPlistpp=[-1,1,1,1,-1,-1,1,-1,1,-1,-1,1]

display(pullMove2D(10,8,estpp,HPlistpp)[1])

10×10 Matrix{Int8}:
 0  0   0  0  0   0   0  0  0  0
 0  0   0  0  0   0   0  0  0  0
 0  0   0  0  0   0   0  0  0  0
 0  0   0  0  0  -1   1  0  0  0
 0  0   0  0  1  -1  -1  0  0  0
 0  0  -1  1  1  -1   1  0  0  0
 0  0   0  0  0  -1   0  0  0  0
 0  0   0  0  0   1   0  0  0  0
 0  0   0  0  0   0   0  0  0  0
 0  0   0  0  0   0   0  0  0  0

10×10 Matrix{Int8}:
 0  0   0  0  0   0   0   0  0  0
 0  0   0  0  0   0   0   0  0  0
 0  0   0  0  0   0   0   0  0  0
 0  0   0  0  0   0   0   0  0  0
 0  0   0  0  1  -1  -1   1  0  0
 0  0  -1  1  1  -1   1  -1  0  0
 0  0   0  0  0  -1   0   0  0  0
 0  0   0  0  0   1   0   0  0  0
 0  0   0  0  0   0   0   0  0  0
 0  0   0  0  0   0   0   0  0  0